# B. From Tagged Data_ToVisualizationFormat

This notebook receives a dataset from DEEP, preprocess it organizing it by Name of Community, add secondary data sets, and export three datasets (descriptors, descriptors_actions, descriptors_vca), that are ready to use in tableau.

In [1]:
%load_ext autoreload
%autoreload 2

import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import matplotlib.pyplot as plt
%matplotlib inline  

import re
import pandas as pd
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn
import pyLDAvis.gensim

#Vader
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.cluster import KMeans

## DEEP extracted data analysis

In [2]:
data_folder = 'data/data_DEEP/Asia Pacific/Philippines/'
excerpts_file = '20190604_DEEP_Entries_Export.xlsx'
excerpts_DEEP = pd.read_excel(data_folder + excerpts_file,encoding="utf-8")
excerpts_DEEP.head(5)

C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWar

,Date of Lead Publication,Imported By,Date Imported,Lead Title,Source,Assignee,Excerpt,Information date,VCA tools used,Livelihood Activities,...,Geophysical environment,Hazards - Dimension,Hazards - Subdimension,Hazards - Sector,Hazards - Subsectors,Considers climate change,VCA Sections - Dimension,VCA Sections - Subdimension,Hazards,Contains good practices/lessons learned
0,2014-12-30,Paola Yela,2019-04-09,Villa Imelda PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Villa Imelda,NaN,"Historical profile, Mapping, Seasonal calendar...","Agriculture, Livestock, Fishery and Aquacultur...",...,"Lowland, Mountain",NaN,NaN,NaN,NaN,No,NaN,NaN,"Cyclones, Other, Floods, Landslides, Fires",No
1,2014-12-30,Paola Yela,2019-04-24,Villa Imelda PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Houses near the lake,NaN,NaN,NaN,...,NaN,Vulnerability,Physical (Infrastructures and services),Floods,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-12-30,Paola Yela,2019-04-24,Villa Imelda PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,We only get help from barangay officials,NaN,NaN,NaN,...,NaN,Vulnerability,Social,Floods,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-12-30,Paola Yela,2019-04-24,Villa Imelda PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Few families living near the lake neglect the ...,NaN,NaN,NaN,...,NaN,Vulnerability,Human,Floods,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-12-30,Paola Yela,2019-04-24,Villa Imelda PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,The BLGU do not have a contingency plan in cas...,NaN,NaN,NaN,...,NaN,Vulnerability,Social,Fires,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# finding community refered in each report 
## Name of community are the excerpts, that are neither describing hazards nor vca sections 
##This change according to the framework
excerpts_DEEP['Name of Community'] = excerpts_DEEP['Excerpt'][pd.isna(excerpts_DEEP['Hazards - Dimension'])*pd.isna(excerpts_DEEP['VCA Sections - Dimension'])]
# replicating name for all entries of report (approximation that assumes all entries of a report are contiguos)
excerpts_DEEP['Name of Community'] = excerpts_DEEP['Name of Community'].replace(0,np.nan).ffill()
excerpts_DEEP['Province'] = excerpts_DEEP['Province'].replace(0,np.nan).ffill()
excerpts_DEEP['Municipality'] = excerpts_DEEP['Municipality'].replace(0,np.nan).ffill()

#adding country name (it should have been added from framework :S)
excerpts_DEEP['Country'] = 'Philippines'

#adding indicator variables of gender disaggregation 
excerpts_DEEP['Disaggregate gender'] = np.logical_or(~pd.isna(excerpts_DEEP['Number of inhabitants - Men']), ~pd.isna(excerpts_DEEP['Number of inhabitants - Women']))

#adding indicator variables of age disaggregation
excerpts_DEEP['Disaggregate age'] = np.logical_or(~pd.isna(excerpts_DEEP['Number of inhabitants - Children (below 18 years old)']), ~pd.isna(excerpts_DEEP['Number of inhabitants - Elder people (over 60 years old)']))
excerpts_DEEP.sample(5)

,Date of Lead Publication,Imported By,Date Imported,Lead Title,Source,Assignee,Excerpt,Information date,VCA tools used,Livelihood Activities,...,Hazards - Subsectors,Considers climate change,VCA Sections - Dimension,VCA Sections - Subdimension,Hazards,Contains good practices/lessons learned,Name of Community,Country,Disaggregate gender,Disaggregate age
1520,2015-07-01,Paola Yela,2019-04-25,VCA Report Tigbalogo_2018-10-11_09-01-44.docx,Philippine Red Cross and German Red Cross,Paola Yela,lack of information and awareness,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Tigbalogo,Philippines,False,False
621,2014-12-30,Paola Yela,2019-04-24,79 PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Other houses are no sanitary toilets,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Barangay 79 (Marasbaras),Philippines,False,False
352,2014-12-30,Paola Yela,2019-04-24,Santiago PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,the area is flat surfaced and houses are near ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Santiago,Philippines,False,False
353,2014-12-30,Paola Yela,2019-04-24,Santiago PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,People has less knowledge on what to do if eve...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Santiago,Philippines,False,False
513,2014-12-30,Paola Yela,2019-04-24,Danao PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Families do not have toilets,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Danao,Philippines,False,False


In [4]:
# solving problem with Name of Province
excerpts_DEEP['Province'] = excerpts_DEEP['Province'].replace('NCR, City of Manila, First District','Metropolitan Manila')
excerpts_DEEP['Province'] = excerpts_DEEP['Province'].replace('NCR, Second District','Metropolitan Manila')
excerpts_DEEP['Province'] = excerpts_DEEP['Province'].replace('NCR, Third District','Metropolitan Manila')
excerpts_DEEP['Province'] = excerpts_DEEP['Province'].replace('NCR, Fourth District','Metropolitan Manila')
excerpts_DEEP['Municipality'] = excerpts_DEEP['Municipality'].replace('City of Valenzuela','Valenzuela')
excerpts_DEEP['Municipality'] = excerpts_DEEP['Municipality'].replace('Tondo I / II','Manila')
excerpts_DEEP['Municipality'] = excerpts_DEEP['Municipality'].replace('Caloocan City','Kalookan City')

excerpts_DEEP['Name of Community'] = excerpts_DEEP['Name of Community'].replace('Catungan-Bugarot','Bugarot')
excerpts_DEEP['Name of Community'] = excerpts_DEEP['Name of Community'].replace('Barangay 79 (Marasbaras)','Barangay 79')
excerpts_DEEP['Name of Community'] = excerpts_DEEP['Name of Community'].replace('Barangay 83-C (San Jose)','Barangay 83-C')
excerpts_DEEP['Name of Community'] = excerpts_DEEP['Name of Community'].replace('Barangay 103-A (San Paglaum)','Barangay 103-A')

## 1) General descriptors

In [5]:
# Extracting information of interest for "general trends" 
fields_general = ['Province','Municipality','Name of Community','Hazards', 'Livelihood Activities','Type of context','Geophysical environment','VCA tools used','Source','Identifies areas most exposed to hazards','Considers climate change','Disaggregate gender','Disaggregate age','Contains good practices/lessons learned','Date of Lead Publication','Language of report','Lead Title']

## As climate change was tagged in PRA and VCA (and not in Plan of Actions), this feature is taken as proxy of unique communities
descriptors = pd.DataFrame(excerpts_DEEP['Country'][~pd.isna(excerpts_DEEP['Considers climate change'])])
for field in fields_general:
    descriptors[field] = excerpts_DEEP[field][~pd.isna(excerpts_DEEP['Considers climate change'])]

descriptors.reset_index(drop=True,inplace=True)
descriptors = descriptors.rename(columns = {'Lead Title': 'Report Title'})
descriptors.tail()

,Country,Province,Municipality,Name of Community,Hazards,Livelihood Activities,Type of context,Geophysical environment,VCA tools used,Source,Identifies areas most exposed to hazards,Considers climate change,Disaggregate gender,Disaggregate age,Contains good practices/lessons learned,Date of Lead Publication,Language of report,Report Title
60,Philippines,Antique,Hamtic,Piape I,"Cyclones, Floods, Earthquake, Fires, Road acci...","Agriculture, Livestock, Income generation acti...",Rural,Lowland,"Community factsheet, Direct observation, Trans...",Philippine Red Cross and German Red Cross,Yes,Yes,True,True,Yes,2015-07-01,English,VCA REPORT Piapi 1 with Action Plan_2018-10-20...
61,Philippines,Antique,San Jose,San Pedro,"Cyclones, Earthquake, Diseases, Floods, Road a...","Agriculture, Fishery and Aquaculture, Employme...",Perirural-Periurban,"Coastal, Lowland","Community factsheet, Direct observation, Trans...",Philippine Red Cross and German Red Cross,Yes,Yes,True,True,Yes,2015-07-01,English,VCA REPORT San Pedro_2018-12-18_01-51-43.docx
62,Philippines,Antique,Patnongon,Tigbalogo,"Cyclones, Floods, Earthquake, Fires","Agriculture, Livestock, Forestry, Income gener...",Rural,"Mountain, Lowland","Community factsheet, Transect walk, Historical...",Philippine Red Cross and German Red Cross,Yes,Yes,True,True,Yes,2015-07-01,English,VCA Report Tigbalogo_2018-10-11_09-01-44.docx
63,Philippines,Camarines Norte,Vinzons,Sabang,"Cyclones, Floods, Drought, Diseases, Fires","Agriculture, Fishery and Aquaculture, Forestry",Rural,"Coastal, Lowland","Mapping, Direct observation, Historical profil...",Philippine Red Cross and Spanish Red Cross,Yes,Yes,False,False,Yes,2015-05-20,English,VCA Sabang_Vinzons_2019-02-07_06-08-08.doc
64,Philippines,Antique,Patnongon,Samalague,"Earthquake, Cyclones, Floods, Diseases, Volcan...",Agriculture,Rural,"Mountain, Lowland","Community factsheet, Direct observation, Trans...",Philippine Red Cross and German Red Cross,Yes,Yes,True,True,Yes,2015-07-01,English,VCA Samalague_2018-10-20_13-46-53.docx


## 1.0) On URL of report

In [6]:
secondary_folder = 'data/data_secondary/Asia Pacific/Philippines/'
id_folder = 'id repository/'
excerpts_file = 'id_repository.xlsx' 
id_repository = pd.read_excel(secondary_folder+id_folder+excerpts_file)
id_repository.sample(5)

C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWar

,Country,Province,Municipality,Name of Community,Report ID,Planned Action ID
10,Philippines,Leyte,Alangalang,Peñalosa,244,NaN
64,Philippines,Antique,Patnongon,Samalague,236,NaN
0,Philippines,Leyte,Macarthur,Villa Imelda,262,NaN
19,Philippines,Leyte,Tacloban City,Barangay 79,251,NaN
23,Philippines,Camarines Norte,Mercedes,Quinapaguian,323,NaN


In [7]:
id_repository['url_report'] = (list(map(lambda x: 'http://vcarepository.info/api/file/'+str(x), id_repository['Report ID'])))
# Handling NAN values in planned actions
id_repository['Planned Action ID'] = id_repository['Planned Action ID'].fillna(0)
id_repository['Planned Action ID'] = id_repository['Planned Action ID'].astype(int)
id_repository['url_action_plan'] = (list(map(lambda x: ('http://vcarepository.info/api/file/'+str(x)) if x != 0 else np.nan, id_repository['Planned Action ID'])))
id_repository['Planned Action ID'] = id_repository['Planned Action ID'].replace('0', np.nan)

In [8]:
id_repository.iloc[28]

Country                                         Philippines
Province                                Metropolitan Manila
Municipality                                     Valenzuela
Name of Community                                  Malanday
Report ID                                               289
Planned Action ID                                       290
url_report           http://vcarepository.info/api/file/289
url_action_plan      http://vcarepository.info/api/file/290
Name: 28, dtype: object

In [9]:
#Merging all the information
descriptors = pd.merge(descriptors, id_repository,  how='left', on=['Country','Province','Municipality','Name of Community'])

## 1.1) On population count

In [10]:
secondary_folder = 'data/data_secondary/Asia Pacific/Philippines/'
demographics_folder = 'demographics/'
excerpts_file = 'VCA_places.xlsx' #include all barangays in municipalities where VCA were made
population = pd.read_excel(secondary_folder+demographics_folder+excerpts_file)
population.sample(5)

C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\Paola.YELA\AppData\Local\Continuum\anaconda3\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWar

,Country,Province,Municipality,Name of Community,Population
356,Philippines,Leyte,Tacloban City,Barangay 40,237
56,Philippines,Antique,Hamtic,Ingwan-Batangan,989
295,Philippines,Leyte,Mayorga,Camansi,520
822,Philippines,Metropolitan Manila,Manila,Barangay 199,3355
5,Philippines,Aklan,Kalibo,Buswang Old,3247


In [11]:
#Merging all the information
descriptors = pd.merge(descriptors, population,  how='left', on=['Country','Province','Municipality','Name of Community'])

## 1.2) On type of hazards

In [12]:
#all theoric hazards that appear in Framework-essentially from PAPE
hazards = ['Cyclones','Drought','Earthquake','Extreme colds','Floods','Hailstorm','Heatwave','Landslides','Diseases','Technological','Tsunami','Volcano','Fires','Conflict/Violence','El niño/La niña','Road accidents','Other']

In [13]:
for hazard in hazards:
    descriptors[hazard] = descriptors['Hazards'].str.count(hazard)
#avoiding conflicts for "other" option in several fields
descriptors.rename(columns={'Other':'Other Hazards'},inplace=True)

## 1.3) On type of livelihoods

In [14]:
#all theoric livelihoods that appear in Framework
livelihoods = ['Agriculture','Livestock','Forestry','Fishery and Aquaculture','Income generation activities','Employment']

In [15]:
for livelihood in livelihoods:
    descriptors[livelihood] = descriptors['Livelihood Activities'].str.count(livelihood)

## 1.4) On type of context

In [16]:
#all theoric type of context that appear in Framework
contexts = ['Rural','Urban', 'Perirural-Periurban']

In [17]:
for context in contexts:
    descriptors[context] = descriptors['Type of context'].str.count(context)

## 1.5) On geophysical environment

In [18]:
#all theoric geophysical environment that appear in Framework
geophysicals = ['Island','Lowland','River valley','Coastal','Mountain','Wetlands']

In [19]:
for geophysical in geophysicals:
    descriptors[geophysical] = descriptors['Geophysical environment'].str.count(geophysical)

## 1.6) On VCA tools used

In [20]:
#all theoric tools that appear in Framework- provided from EVCA and VCA toolbox
tools = ["Mapping","Transect walk", "Seasonal calendar","Venn diagram","Historical profile","Secondary source review","Community factsheet","Focus group discussion","Interview","Direct Observation","Resilience star","Problem tree","Household-neighbourhood vulnerability assessment","Livelihoods and coping strategies analysis","Institutional and social network analysis","Assessing the capacity of people's organization","Brainstorming","Ranking","The wall method","Community meeting","Working together","Other"]

In [7]:
a = ["Mapping","Transect walk", "Seasonal calendar","Venn diagram","Historical profile","Secondary source review","Community factsheet","Focus group discussion","Interview","Direct Observation","Resilience star","Problem tree","Household-neighbourhood vulnerability assessment","Livelihoods and coping strategies analysis","Institutional and social network analysis","Assessing the capacity of people's organization","Brainstorming","Ranking","The wall method","Community meeting","Working together","Other"]
import pandas as pd 
list(pd.DataFrame(a).replace("",'')[0])

['Mapping',
 'Transect walk',
 'Seasonal calendar',
 'Venn diagram',
 'Historical profile',
 'Secondary source review',
 'Community factsheet',
 'Focus group discussion',
 'Interview',
 'Direct Observation',
 'Resilience star',
 'Problem tree',
 'Household-neighbourhood vulnerability assessment',
 'Livelihoods and coping strategies analysis',
 'Institutional and social network analysis',
 "Assessing the capacity of people's organization",
 'Brainstorming',
 'Ranking',
 'The wall method',
 'Community meeting',
 'Working together',
 'Other']

In [21]:
for tool in tools:
    descriptors[tool] = descriptors['VCA tools used'].str.count(tool)
#avoiding conflicts for "other" option in several fields
descriptors.rename(columns={'Other':'Other Tools'},inplace=True)

## 1.7) On National Societies

In [22]:
nss = ['Philippine Red Cross','Spanish Red Cross','German Red Cross','Finnish Red Cross','American Red Cross','Norwegian Red Cross']

In [23]:
for ns in nss:
    descriptors[ns] = descriptors['Source'].str.count(ns)

## 1.8) Others

In [24]:
descriptors['Identifies areas most exposed to hazards'] = descriptors['Identifies areas most exposed to hazards'].map(dict(Yes=1, No=0))
descriptors['Considers climate change'] = descriptors['Considers climate change'].map(dict(Yes=1, No=0))
descriptors['Contains good practices/lessons learned'] = descriptors['Contains good practices/lessons learned'].map(dict(Yes=1, No=0))

In [25]:
descriptors['Disaggregate gender'] = descriptors['Disaggregate gender']*1
descriptors['Disaggregate age'] = descriptors['Disaggregate age']*1

### Exporting dataset

In [26]:
descriptors.sample(3)

,Country,Province,Municipality,Name of Community,Hazards,Livelihood Activities,Type of context,Geophysical environment,VCA tools used,Source,...,The wall method,Community meeting,Working together,Other Tools,Philippine Red Cross,Spanish Red Cross,German Red Cross,Finnish Red Cross,American Red Cross,Norwegian Red Cross
33,Philippines,Camarines Norte,Santa Elena,Basiad,"Cyclones, Floods, Drought, Landslides","Agriculture, Fishery and Aquaculture",Rural,"Coastal, Mountain","Transect walk, Mapping, Venn diagram, Historic...",Philippine Red Cross and Spanish Red Cross,...,0,0,0,1,1,1,0,0,0,0
53,Philippines,Antique,Hamtic,Calala,"Cyclones, Earthquake, Floods, Diseases, Road a...","Agriculture, Fishery and Aquaculture, Employment",Rural,"Coastal, Lowland","Community factsheet, Direct observation, Trans...",Philippine Red Cross and German Red Cross,...,0,0,0,1,1,0,1,0,0,0
36,Philippines,Aklan,Kalibo,Bachaw Sur,"Cyclones, Diseases, Floods, Drought, Other, Ea...","Employment, Income generation activities, Agri...",Perirural-Periurban,Lowland,"Historical profile, Seasonal calendar, Venn di...",Philippine Red Cross and Finnish Red Cross,...,0,0,0,1,1,0,0,1,0,0


In [27]:
folder_preprocessed = 'data/data_preprocessed/Asia Pacific/Philippines/'
file_descriptors = 'descriptors.xlsx'
descriptors.to_excel(folder_preprocessed+file_descriptors, header=True)

## 2) General descriptors with actions planned

In [28]:
#selecting planned actions and fixing format
data_action = excerpts_DEEP[excerpts_DEEP['VCA Sections - Subdimension']=="Planned action"]
data_action['Excerpt'] = data_action['Excerpt'].dropna(axis=0)
data_action['Excerpt'].replace(regex=True,inplace=True,to_replace=r'\r',value=r' ')
data_action['Excerpt'].replace(regex=True,inplace=True,to_replace=r'\n',value=r' ')
data_action.sample(3)

,Date of Lead Publication,Imported By,Date Imported,Lead Title,Source,Assignee,Excerpt,Information date,VCA tools used,Livelihood Activities,...,Hazards - Subsectors,Considers climate change,VCA Sections - Dimension,VCA Sections - Subdimension,Hazards,Contains good practices/lessons learned,Name of Community,Country,Disaggregate gender,Disaggregate age
1543,2015-07-01,Paola Yela,2019-04-23,VCA Samalague_2018-10-20_13-46-53.docx,Philippine Red Cross and German Red Cross,Paola Yela,Provision of livelihood,NaN,NaN,NaN,...,NaN,NaN,VCA sections,Planned action,NaN,NaN,Samalague,Philippines,False,False
19,2015-08-12,Paola Yela,2019-04-17,VCA Action Plan MALANDAY_2018-12-18_01-54-55.docx,Philippine Red Cross,Paola Yela,Iwas Baha,NaN,NaN,NaN,...,NaN,NaN,VCA sections,Planned action,NaN,NaN,Malanday,Philippines,False,False
1326,2015-07-01,Paola Yela,2019-04-17,VCA BARIRI WITH VCA PLAN_2018-12-18_01-42-01.docx,Philippine Red Cross and German Red Cross,Paola Yela,Purchase of Megaphones and hand set radio,NaN,NaN,NaN,...,NaN,NaN,VCA sections,Planned action,NaN,NaN,Bariri,Philippines,False,False


In [29]:
#merging all excerpts (planned actions) of one community in one list
data_action = data_action.groupby(['Country','Province','Municipality','Name of Community'])['Excerpt'].apply(list)
data_action = data_action.reset_index(level=[0,1,2,3])
data_action.sample(3)

,Country,Province,Municipality,Name of Community,Excerpt
6,Philippines,Aklan,Tangalan,Panayakan,[CONDUCT TRAININGS AND SEMINAR for ALTERNATIVE...
34,Philippines,Leyte,Mayorga,Wilson,[5 water tanks installed and functional ...
16,Philippines,Antique,San Jose,San Pedro,"[Declogging/ excavation of drainage canals, Co..."


In [30]:
data_action['Number of planned actions'] = np.zeros(len(data_action))
data_action['Action 1'] = np.empty(len(data_action))
data_action['Action 2'] = np.empty(len(data_action))
data_action['Action 3'] = np.empty(len(data_action))

#selecting three random planned actions per community & counting number of actions planned by community
for i in range(len(data_action)):
    length = len(data_action['Excerpt'][i])
    if (length>=3):
        r = np.random.choice(length, 3,replace= False)
        data_action['Number of planned actions'][i] = length
        data_action['Action 1'][i] = data_action['Excerpt'][i][r[0]]
        data_action['Action 2'][i] = data_action['Excerpt'][i][r[1]]
        data_action['Action 3'][i] = data_action['Excerpt'][i][r[2]]
        
    elif (length==2):
        data_action['Number of planned actions'][i] = 2
        data_action['Action 1'][i] = data_action['Excerpt'][i][0]
        data_action['Action 2'][i] = data_action['Excerpt'][i][1]
        data_action['Action 3'][i] = np.nan
    else:
        data_action['Number of planned actions'][i] = 1
        data_action['Action 1'][i] = data_action['Excerpt'][i][0]
        data_action['Action 2'][i] = np.nan
        data_action['Action 3'][i] = np.nan
#set to 0, NA 
data_action['Number of planned actions'] = data_action['Number of planned actions'].replace(np.nan, 0)

In [31]:
#getting title of document from where planned actions were extracted
data_action1 = excerpts_DEEP[excerpts_DEEP['VCA Sections - Subdimension']=="Planned action"]
data_action1 = data_action1.groupby(['Country','Province','Municipality','Name of Community','Lead Title']).count()
data_action1 = (data_action1.reset_index(level=[0,1,2,3,4])).iloc[:,0:5]
data_action1 = data_action1.rename(columns = {'Lead Title': 'Planned Action Title'})
data_action1.sample(3)

,Country,Province,Municipality,Name of Community,Planned Action Title
1,Philippines,Aklan,Kalibo,Bachaw Sur,Barangay Bakhaw Sur_2018-09-27_05-56-34.docx
39,Philippines,Metropolitan Manila,Kalookan City,Barangay 18,"VCA Assessment Study (Barangay 18, Zone 2, Dis..."
3,Philippines,Aklan,Kalibo,Tinigaw,Barangay Tinigaw_2018-09-26_01-37-44.docx


In [32]:
#Merging actions with title information
data_action = pd.merge(data_action, data_action1, how='left', on=['Country','Province','Municipality','Name of Community'])

In [33]:
#Merging all the information
descriptors_actions = pd.merge(descriptors, data_action,  how='left', on=['Country','Province','Municipality','Name of Community'])

### Exporting dataset

In [34]:
descriptors_actions.sample(3)

,Country,Province,Municipality,Name of Community,Hazards,Livelihood Activities,Type of context,Geophysical environment,VCA tools used,Source,...,German Red Cross,Finnish Red Cross,American Red Cross,Norwegian Red Cross,Excerpt,Number of planned actions,Action 1,Action 2,Action 3,Planned Action Title
21,Philippines,Leyte,Tacloban City,Barangay 83-C,"Floods, Cyclones, Diseases",NaN,Rural,Lowland,"Historical profile, Mapping, Household-neighbo...",Philippine Red Cross and American Red Cross,...,0,0,1,0,[Provided the following alternative livelihood...,9.0,Provide scholarship program,300 toilets constructed and installed,To provide 1120 comfortable houses,83C CAP output.docx
59,Philippines,Antique,Hamtic,Malandog,"Cyclones, Floods, Road accidents, Fires","Agriculture, Fishery and Aquaculture",Rural,"Coastal, Lowland","Community factsheet, Direct observation, Trans...",Philippine Red Cross and German Red Cross,...,1,0,0,0,[Capacity Building on Basic Life support and F...,4.0,Repair and Rehabilitation of Evacuation Center,Capacity Building on Basic Life support and Fi...,Make a Brgy Resolution requesting the Land Tra...,VCA REPORT Malandog with Action plan EDITED_20...
53,Philippines,Antique,Hamtic,Calala,"Cyclones, Earthquake, Floods, Diseases, Road a...","Agriculture, Fishery and Aquaculture, Employment",Rural,"Coastal, Lowland","Community factsheet, Direct observation, Trans...",Philippine Red Cross and German Red Cross,...,1,0,0,0,"[Clearing and ditching of canals, Conduct advo...",2.0,Clearing and ditching of canals,Conduct advocacy training,NaN,VCA REPORT Calala with action plan_2018-10-20_...


In [35]:
folder_preprocessed = 'data/data_preprocessed/Asia Pacific/Philippines/'
file_descriptors = 'descriptors_actions.xlsx'
descriptors_actions.to_excel(folder_preprocessed+file_descriptors, header=True)

## 3) General descriptors with actions planned and vulnerabilities/capacities

### Vulnerabilities

In [36]:
#selecting vulnerabilities and fixing format
data_vulnerabilities = excerpts_DEEP[excerpts_DEEP['Hazards - Dimension']=='Vulnerability']
data_vulnerabilities['Excerpt'] = data_vulnerabilities['Excerpt'].dropna(axis=0)
data_vulnerabilities['Excerpt'].replace(regex=True,inplace=True,to_replace=r'\r',value=r' ')
data_vulnerabilities['Excerpt'].replace(regex=True,inplace=True,to_replace=r'\n',value=r' ')
data_vulnerabilities.sample(3)

,Date of Lead Publication,Imported By,Date Imported,Lead Title,Source,Assignee,Excerpt,Information date,VCA tools used,Livelihood Activities,...,Hazards - Subsectors,Considers climate change,VCA Sections - Dimension,VCA Sections - Subdimension,Hazards,Contains good practices/lessons learned,Name of Community,Country,Disaggregate gender,Disaggregate age
1375,2015-07-01,Paola Yela,2019-04-25,VCA REPORT Caridad with Action Plan EDITED_201...,Philippine Red Cross and German Red Cross,Paola Yela,lack of information and awareness,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Caridad,Philippines,False,False
400,2014-12-30,Paola Yela,2019-04-24,San Isidro PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Households with no comfort rooms dispose waste...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,San Isidro,Philippines,False,False
829,2015-05-31,Paola Yela,2019-05-01,VCA Quinapaguian_Mercedes_2019-02-07_06-05-43....,Philippine Red Cross and Spanish Red Cross,Paola Yela,houses made of light materials,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Quinapaguian,Philippines,False,False


In [37]:
vulnerabilities = pd.DataFrame(data_vulnerabilities["Country"])
vulnerabilities['Province'] = data_vulnerabilities["Province"]
vulnerabilities['Municipality'] = data_vulnerabilities["Municipality"]
vulnerabilities['Name of Community'] = data_vulnerabilities["Name of Community"]
vulnerabilities['Vulnerability'] = data_vulnerabilities["Excerpt"]
vulnerabilities['HazardV'] = data_vulnerabilities["Hazards - Sector"]
vulnerabilities['PillarV'] = data_vulnerabilities["Hazards - Subdimension"]
vulnerabilities.reset_index(inplace=True, drop=True)
vulnerabilities.head(3)

,Country,Province,Municipality,Name of Community,Vulnerability,HazardV,PillarV
0,Philippines,Leyte,Macarthur,Villa Imelda,Houses near the lake,Floods,Physical (Infrastructures and services)
1,Philippines,Leyte,Macarthur,Villa Imelda,We only get help from barangay officials,Floods,Social
2,Philippines,Leyte,Macarthur,Villa Imelda,Few families living near the lake neglect the ...,Floods,Human


In [38]:
#merging all excerpts (vulnerabilities) of one community in one list
vulnerabilities_excerpt = data_vulnerabilities.groupby(['Country','Province','Municipality','Name of Community'])['Excerpt'].apply(list)
vulnerabilities_pillar = data_vulnerabilities.groupby(['Country','Province','Municipality','Name of Community'])['Hazards - Subdimension'].apply(list)
vulnerabilities_hazard = data_vulnerabilities.groupby(['Country','Province','Municipality','Name of Community'])['Hazards - Sector'].apply(list)

In [39]:
data_vulnerabilities = pd.DataFrame(dict(vulnerabilities_excerpt = vulnerabilities_excerpt, vulnerabilities_pillar = vulnerabilities_pillar,vulnerabilities_hazard=vulnerabilities_hazard)).reset_index()
data_vulnerabilities.sample(3)

,Country,Province,Municipality,Name of Community,vulnerabilities_excerpt,vulnerabilities_pillar,vulnerabilities_hazard
50,Philippines,Leyte,Mayorga,Union,[Most households in Zone VII experiences flood...,"[Social, Physical (Infrastructures and service...","[Floods, Floods, Floods, Floods, Diseases, Dis..."
44,Philippines,Leyte,Macarthur,San Isidro,[There is no water treatment for all deep well...,"[Physical (Infrastructures and services), Phys...","[Diseases, Diseases, Diseases, Diseases, Disea..."
41,Philippines,Leyte,Macarthur,Kiling,[Some vegetable gardens are located near the l...,"[Physical (Infrastructures and services), Phys...","[Landslides, Landslides, Landslides, Landslide..."


### Capacities

In [40]:
#selecting capacities and fixing format
data_capacities= excerpts_DEEP[excerpts_DEEP['Hazards - Dimension']=='Capacity']
data_capacities['Excerpt'] = data_capacities['Excerpt'].dropna(axis=0)
data_capacities['Excerpt'].replace(regex=True,inplace=True,to_replace=r'\r',value=r' ')
data_capacities['Excerpt'].replace(regex=True,inplace=True,to_replace=r'\n',value=r' ')
data_capacities.sample(3)

,Date of Lead Publication,Imported By,Date Imported,Lead Title,Source,Assignee,Excerpt,Information date,VCA tools used,Livelihood Activities,...,Hazards - Subsectors,Considers climate change,VCA Sections - Dimension,VCA Sections - Subdimension,Hazards,Contains good practices/lessons learned,Name of Community,Country,Disaggregate gender,Disaggregate age
309,2014-12-30,Paola Yela,2019-04-28,Union PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Barangay Tanods help in enforcing safe crossing.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Union,Philippines,False,False
1257,2016-01-01,Paola Yela,2019-04-29,"VCA Assessment Study (Barangay 18, Zone 2, Dis...","Philippine Red Cross, Finnish Red Cross and No...",Paola Yela,Trained Personnel,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Barangay 18,Philippines,False,False
735,2014-12-30,Paola Yela,2019-04-29,37 A PRA Outputs landscape.docx,Philippine Red Cross and American Red Cross,Paola Yela,Leaders of people's organizations are active i...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Barangay 37-A,Philippines,False,False


In [41]:
capacities = pd.DataFrame(data_capacities["Country"])
capacities['Province'] = data_capacities["Province"]
capacities['Municipality'] = data_capacities["Municipality"]
capacities['Name of Community'] = data_capacities["Name of Community"]
capacities['Capacity'] = data_capacities["Excerpt"]
capacities['HazardC'] = data_capacities["Hazards - Sector"]
capacities['PillarC'] = data_capacities["Hazards - Subdimension"]
capacities.reset_index(inplace=True, drop=True)
capacities.head(3)

,Country,Province,Municipality,Name of Community,Capacity,HazardC,PillarC
0,Philippines,Leyte,Macarthur,Villa Imelda,Houses that has second floors can be utilized ...,Floods,Physical (Infrastructures and services)
1,Philippines,Leyte,Macarthur,Villa Imelda,Barangay officials are active in the community,Floods,Social
2,Philippines,Leyte,Macarthur,Villa Imelda,Some people in the community are eager to solv...,Floods,Human


In [42]:
#merging all excerpts (capacities) of one community in one list
capacities_excerpt = data_capacities.groupby(['Country','Province','Municipality','Name of Community'])['Excerpt'].apply(list)
capacities_pillar = data_capacities.groupby(['Country','Province','Municipality','Name of Community'])['Hazards - Subdimension'].apply(list)
capacities_hazard = data_capacities.groupby(['Country','Province','Municipality','Name of Community'])['Hazards - Sector'].apply(list)

In [43]:
data_capacities = pd.DataFrame(dict(capacities_excerpt = capacities_excerpt, capacities_pillar = capacities_pillar,capacities_hazard=capacities_hazard)).reset_index()
data_capacities.sample(3)

,Country,Province,Municipality,Name of Community,capacities_excerpt,capacities_pillar,capacities_hazard
29,Philippines,Camarines Norte,Santa Elena,Polungguitguit,"[TRAINED PERSONNEL ON DRR, MONITORING CENTER, ...","[Social, Physical (Infrastructures and service...","[Cyclones, Cyclones, Cyclones, Cyclones, Cyclo..."
3,Philippines,Aklan,Kalibo,Tinigaw,"[Trained rescuer, Rescue equipment, RC 143 Vol...","[Human, Physical (Infrastructures and services...","[Cyclones, Floods, Cyclones, Floods, Cyclones,..."
51,Philippines,Leyte,Mayorga,Wilson,[there is an accessible higher areas or upland...,"[Natural environment, Human, Social, Social, C...","[Floods, Floods, Fires, Floods, Fires, Fires, ..."


In [44]:
#Merging all the information
descriptors_vca = pd.merge(descriptors_actions, vulnerabilities,  how='left', on=['Country','Province','Municipality','Name of Community'])
descriptors_vca = pd.merge(descriptors_vca, capacities,  how='left', on=['Country','Province','Municipality','Name of Community'])

### Exporting dataset

In [45]:
descriptors_vca.sample(3)

,Country,Province,Municipality,Name of Community,Hazards,Livelihood Activities,Type of context,Geophysical environment,VCA tools used,Source,...,Action 1,Action 2,Action 3,Planned Action Title,Vulnerability,HazardV,PillarV,Capacity,HazardC,PillarC
4492,Philippines,Leyte,Tacloban City,Barangay 103-A,"Earthquake, Landslides, Floods, Diseases, Othe...","Agriculture, Livestock, Forestry",Rural,Mountain,"Historical profile, Mapping, Household-neighbo...",Philippine Red Cross and American Red Cross,...,300 linear meter grouted riprap constructed,Provided necessary supplies and capital for th...,Irrigation system constructed,103 A CAP output.docx,Some portions of the barangay are situated in ...,Drought,Physical (Infrastructures and services),People are helping each other,Fires,Physical (Infrastructures and services)
850,Philippines,Leyte,Mayorga,San Roque,"Cyclones, Earthquake, Floods, Diseases, Fires,...","Agriculture, Forestry, Livestock, Fishery and ...",Rural,"Coastal, Lowland","Historical profile, Mapping, Household-neighbo...",Philippine Red Cross and American Red Cross,...,16 ft. culvert installed,1 school building with two classrooms constructed,300 houses in Barangay San Roque reconstructed...,San Roque CAP output.docx,There are no designated evacuation area.,Cyclones,Physical (Infrastructures and services),Barangay officials spearhead every evacuation ...,Cyclones,Social
1300,Philippines,Leyte,Alangalang,Hubang,"Floods, Cyclones, Diseases, El niño/La niña","Agriculture, Forestry",Rural,Lowland,"Historical profile, Mapping, Household-neighbo...",Philippine Red Cross and American Red Cross,...,Constructed core shelter for 220 totally damag...,Constructed Drainage for: - Proper 1 - P...,150 Households have accessed to latrines,Hubang CAP output.docx,The people has insufficient knowledge and tech...,Floods,Social,people are enthusiastic on taking the medicine...,Diseases,Human


In [46]:
folder_preprocessed = 'data/data_preprocessed/Asia Pacific/Philippines/'
file_descriptors = 'vulnerabilities.xlsx'
vulnerabilities.to_excel(folder_preprocessed+file_descriptors, header=True)

In [47]:
folder_preprocessed = 'data/data_preprocessed/Asia Pacific/Philippines/'
file_descriptors = 'capacities.xlsx'
capacities.to_excel(folder_preprocessed+file_descriptors, header=True)

In [48]:
folder_preprocessed = 'data/data_preprocessed/Asia Pacific/Philippines/'
file_descriptors = 'descriptors_vca.xlsx'
descriptors_vca.to_excel(folder_preprocessed+file_descriptors, header=True)

## 1) On Most Common type of context:

In [49]:
#all theoric type of context that appear in Framework
contexts = ["Rural","Urban", "Perirural-Periurban"]

In [50]:
data_context = excerpts_DEEP['Type of context'][~pd.isna(excerpts_DEEP['Type of context'])]
data_context = data_context.reset_index(drop=True)

In [51]:
#aproximation of number of communities (LACKING OF PREPROCESSING, TO STABLISH CORRECTLY THIS NUMBER!)
nbCom = len(data_context)
print("Number of communities that reported tools: ", nbCom)

Number of communities that reported tools:  65


In [52]:
#example of type of context of a community
data_context[8]

'Rural'

In [53]:
#count of each one of the contexts
contexts_count = np.empty([len(contexts)])
for context in contexts:
    contexts_count[contexts.index(context)] = data_context.str.count(context).sum()

In [54]:
#sorted values of tools
contexts_vec = pd.DataFrame(contexts, columns = ["Context"])
contexts_vec['Count of communities'] = contexts_count
contexts_vec = contexts_vec.sort_values(['Count of communities'],ascending=False)
contexts_vec['Percentage'] = (contexts_vec['Count of communities']/nbCom)*100
contexts_vec

,Context,Count of communities,Percentage
0,Rural,53.0,81.538462
1,Urban,7.0,10.769231
2,Perirural-Periurban,5.0,7.692308


## 2) On most common tools:

In [55]:
#tools that are being used in reports
data_tools = excerpts_DEEP['VCA tools used'][~pd.isna(excerpts_DEEP['VCA tools used'])]
data_tools = data_tools.reset_index(drop=True)

In [56]:
#example of tools used by one community
data_tools[8]

'Historical profile, Mapping, Household-neighbourhood vulnerability assessment, Seasonal calendar, Ranking, Other, Problem tree'

In [57]:
#all theoric tools that appear in Framework- provided from EVCA and VCA toolbox
tools = ["Mapping","Transect walk", "Seasonal calendar","Venn diagram","Historical profile","Secondary source review","Community factsheet","Focus group discussion","Interview","Direct Observation","Resilience star","Problem tree","Household-neighbourhood vulnerability assessment","Livelihoods and coping strategies analysis","Institutional and social network analysis","Assessing the capacity of people's organization","Brainstorming","Ranking","The wall method","Community meeting","Working together","Other"]

In [58]:
#count of each one of the tools
tools_count = np.empty([len(tools)])
for tool in tools:
    tools_count[tools.index(tool)] = data_tools.str.count(tool).sum()

In [59]:
#sorted values of tools
tools_vec = pd.DataFrame(tools, columns = ["Tool"])
tools_vec['Count of communities'] = tools_count
tools_vec = tools_vec.sort_values(['Count of communities'],ascending=False)
tools_vec['Percentage'] = (tools_vec['Count of communities']/nbCom)*100
#display tools used by more than 75% of the communities
tools_vec[tools_vec['Count of communities']>nbCom*0.75]

,Tool,Count of communities,Percentage
21,Other,62.0,95.384615
17,Ranking,62.0,95.384615
4,Historical profile,61.0,93.846154
11,Problem tree,61.0,93.846154
2,Seasonal calendar,61.0,93.846154
0,Mapping,60.0,92.307692


## 3) On most common livelihoods:

In [60]:
#Livelihoods reported by communities
data_livelihoods = excerpts_DEEP['Livelihood Activities'][~pd.isna(excerpts_DEEP['Livelihood Activities'])]
data_livelihoods = data_livelihoods.reset_index(drop=True)

In [61]:
#example of livelihoods in a community
data_livelihoods[27]

'Employment, Fishery and Aquaculture'

In [62]:
#all theoric livelihoods that appear in Framework
livelihoods = ["Agriculture","Livestock","Forestry","Fishery and Aquaculture","Income generation activities","Employment"]

In [63]:
#count of each one of the livelihoods
livelihoods_count = np.empty([len(livelihoods)])
for livelihood in livelihoods:
    livelihoods_count[livelihoods.index(livelihood)] = data_livelihoods.str.count(livelihood).sum()

In [64]:
#sorted values of livelihoods
livelihoods_vec = pd.DataFrame(livelihoods, columns = ["Livelihood"])
livelihoods_vec['Count of communities'] = livelihoods_count
livelihoods_vec = livelihoods_vec.sort_values(['Count of communities'],ascending=False)
livelihoods_vec['Percentage'] = (livelihoods_vec['Count of communities']/nbCom)*100
livelihoods_vec

,Livelihood,Count of communities,Percentage
0,Agriculture,56.0,86.153846
2,Forestry,35.0,53.846154
1,Livestock,32.0,49.230769
3,Fishery and Aquaculture,30.0,46.153846
4,Income generation activities,29.0,44.615385
5,Employment,15.0,23.076923


## 4) On most common geophysical environment:

In [65]:
#Geophysical environment that are being used in reports
data_geophysical = excerpts_DEEP['Geophysical environment'][~pd.isna(excerpts_DEEP['Geophysical environment'])]
data_geophysical = data_geophysical.reset_index(drop=True)

In [66]:
#example of geophysical environment in a community
data_geophysical.sample(2)

20        River valley
58    Coastal, Lowland
Name: Geophysical environment, dtype: object

In [67]:
#all theoric geophysical environment that appear in Framework
geophysicals = ["Island","Lowland","River valley","Coastal","Mountain","Wetlands"]

In [68]:
#count of each one of the geophysical environments
geophysicals_count = np.empty([len(geophysicals)])
for geophysical in geophysicals:
    geophysicals_count[geophysicals.index(geophysical)] = data_geophysical.str.count(geophysical).sum()

In [69]:
#sorted values of geophysical environment
geophysicals_vec = pd.DataFrame(geophysicals, columns = ["Geophysical environment"])
geophysicals_vec['Count of communities'] = geophysicals_count
geophysicals_vec = geophysicals_vec.sort_values(['Count of communities'],ascending=False)
geophysicals_vec['Percentage'] = (geophysicals_vec['Count of communities']/nbCom)*100
geophysicals_vec

,Geophysical environment,Count of communities,Percentage
1,Lowland,46.0,70.769231
3,Coastal,19.0,29.230769
4,Mountain,17.0,26.153846
0,Island,6.0,9.230769
2,River valley,1.0,1.538462
5,Wetlands,0.0,0.000000


## 5) On most common hazards:

In [70]:
#hazards reported 
data_hazards = excerpts_DEEP['Hazards'][~pd.isna(excerpts_DEEP['Type of context'])]
data_hazards = data_hazards.reset_index(drop=True)

In [71]:
#example of geophysical environment in a community
data_hazards.sample(2)

35    Drought, Floods, Cyclones, Diseases, Earthquake
17                   Cyclones, Other, El niño/La niña
Name: Hazards, dtype: object

In [72]:
#all theoric hazards that appear in Framework-essentially from PAPE
hazards = ["Cyclones","Drought","Earthquake","Extreme colds","Floods","Hailstorm","Heatwave","Landslides","Diseases","Technological","Tsunami","Volcano","Fires","Conflict/Violence","El niño/La niña","Road accidents","Other"]

In [73]:
#count of each one of the hazards
hazards_count = np.empty([len(hazards)])
for hazard in hazards:
    hazards_count[hazards.index(hazard)] = data_hazards.str.count(hazard).sum()

In [74]:
#sorted values of hazards
hazards_vec = pd.DataFrame(hazards, columns = ["Hazard"])
hazards_vec['Count of communities'] = hazards_count
hazards_vec = hazards_vec.sort_values(['Count of communities'],ascending=False)
#display hazards reported by more than 75% of the communities 
hazards_vec[hazards_vec['Count of communities']>nbCom*0.75]
hazards_vec['Percentage'] = (hazards_vec['Count of communities']/nbCom)*100
#hazards_vec[hazards_vec['Count of communities']>len(data_hazards)*0.75]
hazards_vec

,Hazard,Count of communities,Percentage
0,Cyclones,62.0,95.384615
4,Floods,57.0,87.692308
12,Fires,31.0,47.692308
8,Diseases,31.0,47.692308
2,Earthquake,29.0,44.615385
1,Drought,25.0,38.461538
16,Other,22.0,33.846154
7,Landslides,16.0,24.615385
14,El niño/La niña,12.0,18.461538
11,Volcano,9.0,13.846154


## 6) On National Societies involved:

In [75]:
#NS involved in elaboration of projects/reports
#mask for analyze communities rather than documents (there are some times two documents by a community)
data_ns = excerpts_DEEP['Source'][~pd.isna(excerpts_DEEP['Type of context'])]
data_ns = data_ns.reset_index(drop=True)

In [76]:
#example of NS involved
data_ns.sample(2)

13    Philippine Red Cross and American Red Cross
39     Philippine Red Cross and Finnish Red Cross
Name: Source, dtype: object

In [77]:
#dataframe with national societies involved
nss_vec = pd.DataFrame(data_ns.value_counts())
nss_vec['Percentage'] = (nss_vec['Source']/len(data_ns))*100
nss_vec['Supporter National Society'] = nss_vec.index.values
nss_vec = nss_vec.reset_index(drop=True)
nss_vec

,Source,Percentage,Supporter National Society
0,22,33.846154,Philippine Red Cross and American Red Cross
1,15,23.076923,Philippine Red Cross and German Red Cross
2,12,18.461538,Philippine Red Cross and Spanish Red Cross
3,9,13.846154,Philippine Red Cross and Finnish Red Cross
4,4,6.153846,"Philippine Red Cross, Finnish Red Cross and No..."
5,2,3.076923,Philippine Red Cross
6,1,1.538462,Philippine Red Cross and Red Cross 143


In [78]:
nss_vec['Supporter National Society'].replace(regex=True,inplace=True,to_replace=r'Philippine Red Cross and',value=r'')
nss_vec['Supporter National Society'].replace(regex=True,inplace=True,to_replace=r'Philippine Red Cross,',value=r'')
nss_vec['Supporter National Society'].replace(regex=True,inplace=True,to_replace=r'Philippine Red Cross',value=r'')
nss_vec

,Source,Percentage,Supporter National Society
0,22,33.846154,American Red Cross
1,15,23.076923,German Red Cross
2,12,18.461538,Spanish Red Cross
3,9,13.846154,Finnish Red Cross
4,4,6.153846,Finnish Red Cross and Norwegian Red Cross
5,2,3.076923,
6,1,1.538462,Red Cross 143


## 7) On relation hazards/livelihoods/action plans: 

In [79]:
predictors = excerpts_DEEP['Excerpt'][~pd.isna(excerpts_DEEP['Type of context'])]
predictors = pd.DataFrame(predictors)
predictors.rename(columns ={'Excerpt':'Name of community'}, inplace=True)
predictors['Lead Title'] = excerpts_DEEP['Lead Title'][~pd.isna(excerpts_DEEP['Type of context'])]
predictors['Hazards'] = excerpts_DEEP['Hazards'][~pd.isna(excerpts_DEEP['Type of context'])]
predictors['Livelihood Activities'] = excerpts_DEEP['Livelihood Activities'][~pd.isna(excerpts_DEEP['Type of context'])]
predictors = predictors.reset_index()
predictors.head(5)

,index,Name of community,Lead Title,Hazards,Livelihood Activities
0,0,Villa Imelda,Villa Imelda PRA Outputs landscape.docx,"Cyclones, Other, Floods, Landslides, Fires","Agriculture, Livestock, Fishery and Aquacultur..."
1,21,Wilson,Wilson PRA outputs Landscape.docx,"Cyclones, Floods, Diseases, Fires","Forestry, Agriculture, Livestock"
2,277,Union,Union PRA Outputs landscape.docx,"Cyclones, Floods, Drought, Diseases, Road acci...","Agriculture, Forestry, Livestock, Fishery and ..."
3,311,Talisay,Talisay PRA Outputs Landscape.docx,"Cyclones, Floods, Diseases, El niño/La niña","Forestry, Agriculture, Livestock"
4,324,Santa Isabel,Sta. Isabel PRA Outputs landscape.docx,"Conflict/Violence, Cyclones, Drought, Other, F...","Livestock, Agriculture, Forestry"


In [80]:
#count of each one of the hazards for each community
a = np.zeros(shape =(len(predictors['Hazards']),len(hazards)))
hazards_communities = pd.DataFrame(a,columns=hazards)
#count of each one of the hazards in each one of the communities
for community in range(len(predictors['Hazards'])):
    for hazard in range(len(hazards)):
        hazards_communities[hazards[hazard]][community] = str(predictors['Hazards'][community]).count(hazards[hazard])
hazards_communities.head(5)

,Cyclones,Drought,Earthquake,Extreme colds,Floods,Hailstorm,Heatwave,Landslides,Diseases,Technological,Tsunami,Volcano,Fires,Conflict/Violence,El niño/La niña,Road accidents,Other
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [81]:
#count of each one of the hazards for each community
a = np.zeros(shape =(len(predictors['Livelihood Activities']),len(livelihoods)))
livelihoods_communities = pd.DataFrame(a,columns=livelihoods)
#count of each one of the hazards in each one of the communities
for community in range(len(predictors['Livelihood Activities'])):
    for livelihood in range(len(livelihoods)):
        livelihoods_communities[livelihoods[livelihood]][community] = str(predictors['Livelihood Activities'][community]).count(livelihoods[livelihood])

livelihoods_communities.head(5)

,Agriculture,Livestock,Forestry,Fishery and Aquaculture,Income generation activities,Employment
0,1.0,1.0,1.0,1.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,0.0
3,1.0,1.0,1.0,0.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0,0.0


In [82]:
#predictors00 = predictors.join(livelihoods_communities.join(hazards_communities, how='inner'), how='inner')
predictors00 = livelihoods_communities.join(hazards_communities, how='inner')
predictors00.head(5)

,Agriculture,Livestock,Forestry,Fishery and Aquaculture,Income generation activities,Employment,Cyclones,Drought,Earthquake,Extreme colds,...,Landslides,Diseases,Technological,Tsunami,Volcano,Fires,Conflict/Violence,El niño/La niña,Road accidents,Other
0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [83]:
predictors[livelihoods_communities['Agriculture']==1.0]

,index,Name of community,Lead Title,Hazards,Livelihood Activities
0,0,Villa Imelda,Villa Imelda PRA Outputs landscape.docx,"Cyclones, Other, Floods, Landslides, Fires","Agriculture, Livestock, Fishery and Aquacultur..."
1,21,Wilson,Wilson PRA outputs Landscape.docx,"Cyclones, Floods, Diseases, Fires","Forestry, Agriculture, Livestock"
2,277,Union,Union PRA Outputs landscape.docx,"Cyclones, Floods, Drought, Diseases, Road acci...","Agriculture, Forestry, Livestock, Fishery and ..."
3,311,Talisay,Talisay PRA Outputs Landscape.docx,"Cyclones, Floods, Diseases, El niño/La niña","Forestry, Agriculture, Livestock"
4,324,Santa Isabel,Sta. Isabel PRA Outputs landscape.docx,"Conflict/Violence, Cyclones, Drought, Other, F...","Livestock, Agriculture, Forestry"
5,340,Santiago,Santiago PRA Outputs landscape.docx,"Floods, Earthquake, Cyclones, Other","Agriculture, Forestry"
6,370,San Roque,San Roque PRA Outputs landscape.docx,"Cyclones, Earthquake, Floods, Diseases, Fires,...","Agriculture, Forestry, Livestock, Fishery and ..."
7,395,San Isidro,San Isidro PRA Outputs landscape.docx,"Floods, Earthquake, Cyclones, Diseases","Agriculture, Livestock, Forestry"
8,420,San Antonio,San Antonio PRA Outputs landscape.docx,"Cyclones, Conflict/Violence, Floods, Landslides","Agriculture, Forestry"
9,430,Oguisan,Oguisan PRA Outputs landscape.docx,"Cyclones, Conflict/Violence, Landslides","Forestry, Agriculture"


### 7.1) k-means approach for clustering:

In [84]:
#want to group communities by livelihood and hazards criteria
kmeans = KMeans(n_clusters=2, random_state=0).fit(predictors00)

#### On groups:

In [85]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0])

In [86]:
#communities in group 1
group1 = predictors['Name of community'][kmeans.labels_==0]
group1

6                    San Roque
16                Barangay 105
21    Barangay 83-C (San Jose)
25                Pinagtigasan
28                    Malanday
30             Hen. T. De Leon
34                    Banocboc
35                Bachaw Norte
36                  Bachaw Sur
38                      Jawili
41                   Poblacion
42                       Tagas
43                     Tinigaw
45                 Barangay 32
46                 Barangay 18
47                 Barangay 35
48                 Barangay 73
49                      Bariri
50                    Cuyapiao
51                   Poblacion
52                     Caridad
53                      Calala
54            Catungan-Bugarot
57                    Inabasan
58                      La Paz
59                    Malandog
60                     Piape I
61                   San Pedro
62                   Tigbalogo
64                   Samalague
Name: Name of community, dtype: object

In [87]:
#communities in group 2
group2= predictors['Name of community'][kmeans.labels_==1]
group2

0                     Villa Imelda
1                           Wilson
2                            Union
3                          Talisay
4                     Santa Isabel
5                         Santiago
7                       San Isidro
8                      San Antonio
9                          Oguisan
10                        Peñalosa
11                         Burabod
12                          Kiling
13                          Hubang
14                           Danao
15                          Burgos
17                           Aslum
18                         Astorga
19        Barangay 79 (Marasbaras)
20                   Barangay 37-A
22    Barangay 103-A (San Paglaum)
23                    Quinapaguian
24                  Polungguitguit
26                       Manguisoc
27                     Mangcawayan
29                       Kagtalaba
31                      Cayucyucan
32                         Caringo
33                          Basiad
37                  

In [88]:
print("Number of communities in group 0: ", nbCom-kmeans.labels_.sum())

Number of communities in group 0:  30


#### On behavior of groups:

In [89]:
kmeans.cluster_centers_

array([[0.7       , 0.3       , 0.16666667, 0.53333333, 0.63333333,
        0.4       , 0.96666667, 0.26666667, 0.73333333, 0.        ,
        1.        , 0.        , 0.        , 0.16666667, 0.53333333,
        0.        , 0.06666667, 0.2       , 0.76666667, 0.03333333,
        0.1       , 0.16666667, 0.36666667],
       [1.        , 0.65714286, 0.85714286, 0.4       , 0.28571429,
        0.08571429, 0.94285714, 0.48571429, 0.2       , 0.        ,
        0.77142857, 0.        , 0.        , 0.31428571, 0.42857143,
        0.        , 0.17142857, 0.08571429, 0.22857143, 0.2       ,
        0.25714286, 0.02857143, 0.31428571]])

In [90]:
#difference between both groups
np.absolute(kmeans.cluster_centers_[1]-kmeans.cluster_centers_[0])>0.3

array([ True,  True,  True, False,  True,  True, False, False,  True,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False])

In [91]:
##Difference found between both clusters
np.array(list(predictors00))[np.absolute(kmeans.cluster_centers_[1]-kmeans.cluster_centers_[0])>0.3]

array(['Agriculture', 'Livestock', 'Forestry',
       'Income generation activities', 'Employment', 'Earthquake',
       'Fires'], dtype='<U28')